# Assignment module 3


Some code sections are proposed below. Please, complete and adapt each section according to the requirements.

<ins>**Dataset**</ins>
1. [Unit Linked](https://dutangc.github.io/CASdatasets/reference/eusavingsurrender.html)

## Importing packages <a name="Importing_packages"></a>

Execute the following pip install and imports in your command prompt for the environment you are using. Please, try to rely on these as much as possible, as they should be sufficient to carry out the exercise. However, you can choose other packages. In such case, please explain your choice.


In [ ]:
## Perform these pip installs in the command prompt, with your python kernel activated
!pip install shap
!pip install tensorflow
!pip install shapicant
!pip install scikeras
!pip install mlflow
!pip install evidently
!pip install boruta
!pip install pandas scikit-learn deap
!pip install aif360

!git clone https://github.com/Regis0323/Module-3-assignment.git
!pip install scikit-fuzzy

In [7]:
# 1. SETUP: load libraries & data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data preprocesing and models
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

# MLFlow
import mlflow
import cloudpickle
from evidently import Dataset
from evidently import DataDefinition
from evidently import Report
from evidently.presets import DataDriftPreset, DataSummaryPreset

# Feature selection and explainability of the model
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance
from boruta import BorutaPy
from deap import base, creator, tools, algorithms
import random

# Bias mitigation
from aif360.datasets import StandardDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.preprocessing import Reweighing, OptimPreproc
from aif360.algorithms.inprocessing import MetaFairClassifier, AdversarialDebiasing
from aif360.algorithms.postprocessing import RejectOptionClassification, EqOddsPostprocessing



pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


# Data exploration and preoprocessing


In [28]:
# Load
df = pd.read_csv("/content/Module-3-assignment/eusavingULnoPS.csv")
display(df.head())
print(df.dtypes.value_counts())

,policy.ID,issue.date,termination.date,lapse.reason,premium.frequency,gender,underwriting.age,face.amount,risk.premium,saving.premium,...,rate2Y.relvar1mth,rate2Y.relvar1qtr,rate10Y.relvar1mth,rate10Y.relvar1qtr,unemploy.relvar1mth,unemploy.relvar1qtr,industry.relvar1mth,industry.relvar1qtr,RTV.relvar1mth,RTV.relvar1qtr
0,N1,1999-01-01,2006-04-01,Claim,unique,Male,77,2979.53,265.86,29795.27,...,0.086310,0.184211,0.081598,0.144626,-0.011494,-0.011494,-0.000979,0.016699,0.005941,0.006931
1,N2,1999-01-01,2006-04-01,Claim,unique,Male,77,3039.71,271.23,30397.16,...,0.086310,0.184211,0.081598,0.144626,-0.011494,-0.011494,-0.000979,0.016699,0.005941,0.006931
2,N3,1999-01-01,2005-04-01,Surrender,unique,Female,72,2994.12,80.43,35929.44,...,-0.025391,-0.018489,-0.023343,-0.018868,-0.010309,-0.040816,0.001000,0.003024,0.002002,0.000998
3,N4,1999-01-01,2003-02-01,Surrender,unique,Male,35,3051.10,7.50,30511.01,...,-0.041762,-0.133838,-0.038309,-0.039273,0.000000,-0.008850,-0.003067,0.000000,0.009524,0.009574
4,N5,1999-01-01,2008-01-01,In force,unique,Male,40,5810.21,81.39,58102.11,...,0.037712,-0.004430,0.025938,-0.002027,0.022727,0.069767,0.001876,0.000941,-0.011550,-0.026692


float64    23
object      6
int64       1
Name: count, dtype: int64


Our dataset is composed of 30 columns, of which 6 are categorized as 'object', 1 as 64-bit 'integer' (int64) and 23 as 64-bit 'floating-point' (float64) numbers.
<br>
We shall now check for missing and/or duplicated values before treating them accordingly if any.

In [32]:
#Potential missing values
Missing_Values=df.isna().sum()
print(Missing_Values)

#Potential duplicated values
Duplicated_Values=df[df.duplicated(keep=False)]
print(Duplicated_Values)
  #Treating duplicated values (if any)
df_cleaned=df.drop_duplicates(keep='first')

# Treating date columns/variables
Date_Variables = [col for col in df_cleaned.columns if 'date' in col.lower()]
for Date_column in Date_Variables:
    df_cleaned[Date_column] = pd.to_datetime(df_cleaned[Date_column], errors='coerce')

#Information about the new cleaned dataset
df_cleaned.info()
print(df_cleaned.columns.tolist())

policy.ID              0
issue.date             0
termination.date       0
lapse.reason           0
premium.frequency      0
gender                 0
underwriting.age       0
face.amount            0
risk.premium           0
saving.premium         0
CPI.relvar1mth         0
CPI.relvar1qtr         0
CPI.relvar1yr          0
CPI.relvar2yr          0
EUidx.relvar1mth       0
EUidx.relvar1qtr       0
EUidx.relvar1yr        0
EUidx.relvar2yr        0
rate1Y.relvar1mth      0
rate1Y.relvar1qtr      0
rate2Y.relvar1mth      0
rate2Y.relvar1qtr      0
rate10Y.relvar1mth     0
rate10Y.relvar1qtr     0
unemploy.relvar1mth    0
unemploy.relvar1qtr    0
industry.relvar1mth    0
industry.relvar1qtr    0
RTV.relvar1mth         0
RTV.relvar1qtr         0
dtype: int64
Empty DataFrame
Columns: [policy.ID, issue.date, termination.date, lapse.reason, premium.frequency, gender, underwriting.age, face.amount, risk.premium, saving.premium, CPI.relvar1mth, CPI.relvar1qtr, CPI.relvar1yr, CPI.relvar2yr, EUidx.

After this check, we can see that there are neither missing values nor duplicates and there are 2 date variables that have been properly treated.

We shall now create two datasets (df_cat and df_num), one for each of the following two dependent variables:
*   a categorical one: lapse.reason;
*   a numeric one: a numerical variable that represents the proportion of risk premium over total premium: proportion_risk_premium = risk.premium / (risk.premium+saving.premium)

In [48]:
df_cat =df_cleaned['lapse.reason']
df_num = df_cleaned[['risk.premium','saving.premium']]


# Creation of numerical dependent variable
df_num['proportion_risk_premium'] = 0
df_num['proportion_risk_premium'] = df_num['risk.premium']/(df_num['risk.premium'] + df_num['saving.premium'])
df_num=df_num['proportion_risk_premium']

print(df_num.describe())
print('Number of NaN:', df_num.isna().sum())

count    21921.000000
mean         0.003823
std          0.004241
min          0.000000
25%          0.000210
50%          0.001133
75%          0.009900
max          0.020433
Name: proportion_risk_premium, dtype: float64
Number of NaN: 0


## Data preprocessing
For both the numerical and categorical problems, define the X (X_cat and X_num) and y datasets, convert categorical variables to a suitable format (label encoding or one-hot encoding) and handle date variables.

Split the datasets into training and test sets.



In [ ]:
# Define X and y (wisely select variables for the X datasets)
y_num =
y_cat =
X_num =
X_cat =

# Label encoding or one-hot encoding


# Example on how to handle date variables
df_cat['termination.date'] = pd.to_datetime(df_cat['termination.date'], errors='coerce')
# Create new numeric features from dates
df_cat['termination.date.yr'] = df_cat['termination.date'].dt.year
df_cat['termination.date.month'] = df_cat['termination.date'].dt.month

# Split test and train datasets


## Development of model for categorical variable: RandomForest

For the categorical dependent variable "lapse.reason", develop a Random Forest algorithm.

## Development of model for numerical variable (proportion): Sequential Dense NN


For the numerical variable "proportion_risk_premium", develop a Sequential Dense Neural Network.

# Variable selection

## Categorical problem, feature selection


For the categorical dependent variable "lapse.reason", perform Boruta feature selection using (from boruta) BortutaPy.

For the categorical dependent variable "lapse.reason", perform Genetic Algorith feature selection using (from deap) base, creator, tools, algorithms repspectively.

Optional: Use MLFLow to show you know how to use good development and tracking practices while training the model.

In [ ]:
# Setup MLFlow

host =
port =
mlflow.set_tracking_uri(uri=f"http://{host}:{port}")


In [ ]:
# Model packaging


In [ ]:
# Model registry


In [ ]:
# Model serving


In [ ]:
# Model monitoring


## Numerical problem, feature selection

For the numerical variable "proportion_risk_premium", perform a forward and a backward selection on explanatory variables.




For the numerical variable "proportion_risk_premium", apply Lasso using LassoCV.

Optional: Use MLFLow to show you know how to use good development and tracking practices while training the model.

In [ ]:
# Setup MLFlow

host =
port =
mlflow.set_tracking_uri(uri=f"http://{host}:{port}")


In [ ]:
# Model packaging


In [ ]:
# Model registry


In [ ]:
# Model serving


In [ ]:
# Model monitoring


# Explainability of the model

## Categorical problem

Explain the RandomForest model to a client, that wants to know why their quote is so high by permutation feature selection using permutation_importance.

Choose another method of your preference to explain the RandomForest model.

## Numerical problem

Explain the Neural Netwok model to a client, that wants to know why their quote is so high using Shapley values.

Choose another method of your preference to explain the NN model.

# BIAS for the numerical problem

After having set a threshold = 0.0002 = 0.02%, define a variable called "risk_category" that takes values High or Low if variable "proportion_risk_premium" is higher or lower than the treshold.

Similarly, define variable "risk_category_pred" that does the same but on the basis of the values predicted by the Neural Network model. For these two variables, count the number of High and Low values for Females and Males.

In [ ]:
# Threshold = 0.0002 = 0.02%
threshold = 0.0002

df_num['risk_category'] = np.where(df_num['proportion_risk_premium'] > threshold, 'High', 'Low')
df_num['risk_category_pred'] =

Using confusion_matrix, compare real and predicted values by counting False positives, False negatives, Trues positives, True negatives.

## Metrics to detect possible bias on predicted values
Investigate possible bias on the basis of the metrics:


*   Demographic Parity (no bias range: -0.1, 0.1)
*   Equal Opportunity (no bias range: -0.1, 0.1)
*   Average Odds (no bias range: -0.1, 0.1)
*   Disparate Impact (no bias range: 0.8, 1.25)
*   Theil Index (no bias range: 0, 0.15)

## Bias mitigation
Proceed with:

*   Preprocessing bias mitigation: choose the method, between reweighing and optimized preprocessing, that leads to a greater bias reduction
*   In-processing bias mitigation:  choose the method, between Meta Fair
*   Postprocessing bias mitigation: Reject option classification and equalized Odds




In [ ]:
# Preprocessing


In [ ]:
# In-processing


In [ ]:
# Post-Processing


Compare pros and cons of chosen methods:




Write down in text what you think the pros and cons are of the methods above